# LWE Grundlagen

Die Folgenden abschnitte sind zur Beschreibung und einführung in das LWE Problem. Dies ist die Grundlage für verschiedene moderne Verschlüsselungsalgorithmen wie Kyber, aber auch die basis für diverse Homomorphe Algorithmen.

Im folgenden werden 3 Arten von LWE beschrieben und implementiert
- Learning with Errors (LWE)
- Ring-LWE (RLWE)
- Module-LWE (MLWE)

Das LWE verfahren liefert dabei die Grundlagen auf denen die anderen beiden Aufbauen. Bei RLWE wird das LWE verfahren auf einen Polynom Ring übertragen und bei MLWE wird eine Matrix aus Polynom Ringen benutzt. RLWE ist somit ein Spezialfall von MLWE, bei dem eine 1x1 Matrize verwendet wird.

Zur Umsetzung von LWE muss zuerst ein Ring der ganzen zahlen modulo eines Faktors $q$ definiert werden, welcher die addition (+) und multiplikation (*) unterstützt Dies wird definiert als 
$$\mathbb{Z}_q= \mathbb{Z}/q\mathbb{Z}$$
Dies ist somit eigentlich nur die Gruppe aller ganzen Zahlen, welche kleiner als $q$ und größer gleich 0 sind. Beispielsweise $\mathbb{Z}_3 = \{0,1,2\}$

## LWE

$\chi \in \mathbb{Z}_q$ ist eine diskrete normalverteilung mit werten im Ring

Private key: $s\leftarrow \chi^{m}$. Dabei entsteht ein vektor mit länge $m$ bei dem alle werte normalverteilt aus $\mathbb{Z}_q$ stammen.

Public key:
1. erstellen einer Matrix mit gleichmäßig zufälligen werten: 
$A = \begin{bmatrix}
a_11 & \cdots  & a_1n\\
\vdots  & \ddots  & \vdots \\
a_m1 & \cdots & a_mn
\end{bmatrix} \leftarrow \mathbb{Z}_q^{n \times m} $
2. berechnen des vektors $b$ durch: $b = As+e \in \mathbb{Z}_q^{n}$ mit $e \leftarrow \chi^n$ 
3. der private schlüsse ergibt sich aus den zwei Werten: $P = (A, b)$

Zum Verschlüsseln eines binären Wertes $m \in \mathbb{Z}_2$ (entspricht den Werten $\{0, 1\}$) werden zwei Werte Berechnet:
1. $u = A^T \cdot r + e_1 \in \mathbb{Z}_q^m$
2. $v = b^T \cdot r + e_2 + (m*\left\lfloor q/2\right\rfloor) \in \mathbb{Z}_q$

Wobei $r \in \chi^n$ zusätzliche Werte sind um ein zusätzliche Verschleierung zu erzeugen und $e_1 \in \chi^m$ und $e_2 \in \chi$ sind zusätzliche Fehlerwerte. Durch diese drei werte wird jede Verschlüsselung einzigartig und somit schwerer zu entschlüsseln durch das sammeln vieler verschlüsselter Texte. 

Somit ergeben sich zwei werte, der Vektor $u$ und der skalar $v$. Die Nachricht wurde dabei auf skaliert sodas $0 \rightarrow 0$ und $1 \rightarrow \left\lfloor q/2\right\rfloor$. Wenn man sich den Ring $\mathbb{Z}_q$ als Uhr dabei vorstellt (wie es oft für modulo Rechnungen visualisiert wird), dann ist der Nachrichten Wert $0$ nun an der 12Uhr position in der Uhr und der Nachrichten Wert $1$ (ungefähr) an der 6 Uhr Position. Durch die einberechneten Fehler verändert sich der eigentliche Nachrichten Wert um ein kleines bisschen, sodass er quasi mehr in die viertel oder dreiviertel Stellung auf der Uhr zeigt.

Das entschlüsseln der Nachricht erfolgt mithilfe folgender Gleichung:
$$
\begin{align*}
m &= \left\lfloor \frac{1}{\left\lfloor q/2\right\rfloor}*(v-s^T \cdot u)\right\rceil _2 \\
  &= \left\lfloor \frac{1}{\left\lfloor q/2\right\rfloor}*(b^T \cdot r + e_2 + (m*\left\lfloor q/2\right\rfloor)-s^T \cdot (A^T \cdot r + e_1))\right\rceil _2\\
  &= \left\lfloor \frac{1}{\left\lfloor q/2\right\rfloor}*((As+e)^T \cdot r + e_2 + (m*\left\lfloor q/2\right\rfloor)-s^T A^T \cdot r - s^T e_1)\right\rceil _2\\
  &= \left\lfloor \frac{1}{\left\lfloor q/2\right\rfloor}*((As)^T \cdot r + e^Tr+ e_2 + (m*\left\lfloor q/2\right\rfloor)-(As)^T \cdot r - s^T e_1)\right\rceil _2\\
  &= \left\lfloor \frac{1}{\left\lfloor q/2\right\rfloor}*(e^Tr+ e_2 + (m*\left\lfloor q/2\right\rfloor)- s^T e_1)\right\rceil _2\\
  &= \left\lfloor \frac{e^Tr}{\left\lfloor q/2\right\rfloor}+ \frac{e_2 }{\left\lfloor q/2\right\rfloor}+ m - \frac{s^T e_1}{\left\lfloor q/2\right\rfloor}\right\rceil _2\\
  &= \left\lfloor m' \right\rceil _2\\
  &= m \in \{0,1\}
\end{align*}
$$

Bei den letzten zwei schritten wird davon ausgegangen das die Fehlerwerte nahe null sind und somit nur einen geringen einfluss auf $m$ haben. Dadurch werden diese Werte durch das runden wieder ausgeglichen und die Nachricht kommt am ende zum vorschein, welche noch in ihre ring (modulo 2) angepasst werden muss.

Damit dies funktioniert muss: $e^Tr+ e_2 - s^T e_1 < q/4$ sein. Der Fehlerterm muss somit kleiner als ein viertel q sein. Dies lässt sich erneut leicht über das Uhren beispiel erklären. Da die Zeiger am Ende auf die volle bzw halbe Stunde gerundet werden, dürfen die Zeiger nicht über die viertel oder dreiviertel stunde wander, da ansonsten zur falschen stelle gerundet wird. Dies Bewegung entspricht jeweils eine viertel Umdrehung um $q$. Somit darf der Fehlerterm nicht größer als $q/4$ sein. Aus diesem Grund muss darauf geachtet werden das $s, e, e_1, e_2, r$ nicht zu groß sind, damit dies eingehalten werden kann, weshalb diese Werte aus der Normalverteilung herausgezogen werden und nicht uniform aus $\mathbb{Z}_q$

### Beispiel Rechnung

Um die etwas abstrakten Rechnungen einfacher verständlich zu machen folgt hier eine Beispiel Rechnung.
Zuerst werden die Parameter definiert
$$
\begin{align*}
q &= 100 \\
n &= 2 \\
m &= 2 \\
\end{align*}
$$

Die Schlüsselgenerierung (KeyGen)
$$
s = \begin{bmatrix}1 \\ 2 \end{bmatrix}
A = \begin{bmatrix}56 & 77 \\ 29 & 59 \end{bmatrix}
e = \begin{bmatrix}99 \\ 1 \end{bmatrix} \\
\begin{align*}
\\
b &= As+e \\
  &= \begin{bmatrix}56 & 77 \\ 29 & 59 \end{bmatrix}\cdot \begin{bmatrix}1 \\ 2 \end{bmatrix} &+\begin{bmatrix}99 \\ 1 \end{bmatrix}\\
  &= 1 \cdot \begin{bmatrix}56 \\ 29 \end{bmatrix} + 2 \cdot \begin{bmatrix}77 \\ 59 \end{bmatrix} &+ \begin{bmatrix}99 \\ 1 \end{bmatrix} \\
  &= \begin{bmatrix}309 \\ 148 \end{bmatrix}_q \\
  &= \begin{bmatrix}9 \\ 48 \end{bmatrix} \\
\\
Sk &= s \\
Pk &= (A, b) \\
   &= (\begin{bmatrix}56 & 77 \\ 29 & 59 \end{bmatrix}, \begin{bmatrix}9 \\ 48 \end{bmatrix}) \\
\end{align*}
$$

Verschlüsselung der Nachricht $m=1$
$$
r = \begin{bmatrix}0 \\ 2 \end{bmatrix} 
e_1 = \begin{bmatrix}2 \\ 0 \end{bmatrix} 
e_2 = 99 \\
\begin{align*}
\\
u &= A^T \cdot r + e_1 \\
  &= \begin{bmatrix}56 & 77 \\ 29 & 59 \end{bmatrix}^T \cdot \begin{bmatrix}0 \\ 2 \end{bmatrix} &+ \begin{bmatrix}2 \\ 0 \end{bmatrix} \\
  &= \begin{bmatrix}56 & 29 \\ 77 & 59 \end{bmatrix} \cdot \begin{bmatrix}0 \\ 2 \end{bmatrix} &+ \begin{bmatrix}2 \\ 0 \end{bmatrix} \\
  &= 0\cdot \begin{bmatrix}56 \\ 77 \end{bmatrix} + 2 \cdot \begin{bmatrix}29 \\ 59 \end{bmatrix} &+ \begin{bmatrix}2 \\ 0 \end{bmatrix} \\
  &= \begin{bmatrix}60 \\ 118 \end{bmatrix}_q \\
  &= \begin{bmatrix}60 \\ 18 \end{bmatrix} \\
\\
v &= b^T \cdot r + e_2 + (m*\left\lfloor q/2\right\rfloor) \\
  &= \begin{bmatrix}9 \\ 48 \end{bmatrix}^T \cdot \begin{bmatrix}0 \\ 2 \end{bmatrix} + 99 + 1 \cdot \left\lfloor 100/2\right\rfloor \\
  &= \begin{bmatrix}9 & 48 \end{bmatrix} \cdot \begin{bmatrix}0 \\ 2 \end{bmatrix} + 99 + 50 \\
  &= 9 \cdot 0 +48 \cdot 2 + 99 + 50 \\
  &= 245_q \\
  &= 45 \\
\end{align*}

C = (u, v)
$$

Entschlüssel der verschlüsselten Nachricht $C$
$$
\begin{align*}
m &= \left\lfloor \frac{1}{\left\lfloor q/2\right\rfloor} *(v-s^T \cdot u)\right\rceil _2 \\
  &= \left\lfloor \frac{1}{\left\lfloor 100/2\right\rfloor} * (45-\begin{bmatrix}1 \\ 2 \end{bmatrix}^T \cdot \begin{bmatrix}60 \\ 18 \end{bmatrix})\right\rceil _2 \\
  &= \left\lfloor \frac{1}{50} * (45-\begin{bmatrix}1 & 2 \end{bmatrix} \cdot \begin{bmatrix}60 \\ 18 \end{bmatrix})\right\rceil _2 \\
  &= \left\lfloor \frac{1}{50} * (45-(60 \cdot 1 + 18 \cdot 2))\right\rceil _2 \\
  &= \left\lfloor \frac{1}{50} * (-33)_q\right\rceil _2 \\
  &= \left\lfloor \frac{1}{50} * 67\right\rceil _2 \\
  &= \left\lfloor \frac{67}{50}\right\rceil _2 \\
  &= 1 \\
\end{align*}
$$

Somit konnte die Originale Nachricht wiederhergestellt werden

### Beispiel Code

In [9]:
import numpy as np
from numpy.random import randint, normal

def sample_small(n, q, sigma=1.0):
  """Samples small vector from an approximated discrete Gaussian distribution.
  """
  # Sample from continuous Gaussian with mean 0 and std dev sigma
  error = np.round(normal(loc=0, scale=sigma, size=n)).astype(int) % q 

  # if its the zero vector, try again
  if sum(error) == 0:
    return sample_small(n, q, sigma)
  return error

# Parameters
modulus = 100
t = 2 # The message ring
q_half = np.floor(modulus/2)
m_p = 2
m_p = 2

# Creating Privat and Secret Key
s = sample_small(m_p, modulus)
A = randint(0, modulus, (m_p, m_p))
e = sample_small(m_p, modulus)
b = (A@s+e) % modulus

print(f"""Secret Key: {s}
Private Key:
      A={A},
      b={b}
Generated with error: {e}""")

Secret Key: [99 98]
Private Key:
      A=[[64 69]
 [46 27]],
      b=[97  1]
Generated with error: [99  1]


In [10]:
message = randint(0, t) # random 0 or 1 as message
r = sample_small(m_p, modulus)
e1 = sample_small(m_p, modulus)
e2 = sample_small(1, modulus)
u = (A.T@r+e1) % modulus
v = (b.T@r+e2+message*q_half) % modulus

print(f"""Encrypting message: {message}
using:
    r={r},
    e1={e1}
    e2={e2}
Resulting in:
    u={u}
    v={v}
""")
decrypt = np.round((1/q_half)*((v-(s.T@u)) % modulus)) % t
print(f"""Decrypts into message: {decrypt}""")

Encrypting message: 1
using:
    r=[99 99],
    e1=[ 1 99]
    e2=[99]
Resulting in:
    u=[91  3]
    v=[51.]

Decrypts into message: [1.]


## Ring-LWE

Der oben beschriebene Algorithmus soll nun abgewandelt werden, sodass anstatt mit Matrixmultiplikation zu rechnen, ein Polynom-Ring verwendet wird. Dieser wird definiert als 
$$R_q = \mathbb{Z}_q[x]/f$$

Somit werden die einzelnen Variablen nun aus diesem Ring entnommen.

### Beispiel
Als erstes muss dafür eine Polynomfunktion definiert werden. Häufig wird dabei $f = X^n+1$ verwendet. Somit ensteht der Polynomring
$$R_q = \mathbb{Z}_q[x]/(X^n+1)$$
$n$ entspricht dabei dem maximalen Grad des Polynoms.

Wie auch beim ersten mal müssen wir nun die parameter definieren:
$$
\begin{align*}
q &= 100 \\
f &= X^n+1 \\
n &= 3
\end{align*}
$$

Bei der Schlüsselerstellung werden wie auch schon beim Plain-LWE, die Parameter von $s, e$ als kleine Werte aus der diskreten normalverteilung gezogen und $A$ aus dem kompletten Ring:
$$
\begin{align*}
s &= 1 + 0x + 1x^2 \\
A &= 28 + 56x + 1x^2 \\
e &= 1 + 99x + 2x^2\\
\\
b &= As + e \\
  &= (28 + 56x + 1x^2)*(1 + 0x + 1x^2) + (1 + 99x + 2x^2) \\
  &= (28 + 28x^2) + (56x + 56x^3) + (1x^2 + 1x^4) + (1 + 99x + 2x^2) \\
  &= 29 + 155x + 31x^2 + 56x^3 + 1x^4 \mod f\\
  &= 29 + 155x + 31x^2 - 56 - 1x  \\
  &= -27 + 154x + 31x^2  \mod q\\
  &= 73 + 54x + 31x^2
\end{align*}
$$

Der Vorteil beim Verschlüsseln mithilfe von R-LWE ist, das eine Nachricht in Länge des Polynoms verschlüsselt werden kann. Als nächstes soll die Nachricht $m = (1, 1, 0)$ verschlüsselt werden. Zuerst werden die benötigten Parameter initialisiert:

$$
\begin{align*}
r=99 + 99x + 2x^2,
e_1=98 + 0x + 98x^2
e_2=1 + 0x + 0x^2
m = (1, 1, 0) = 1+ 1x+ 0x^2
\end{align*}
$$

Anschließend flogt die Berechnung von $u, v$.
$$
\begin{align*}
u &= A \cdot r + e_1 \\
  &= (28 + 56x + 1x^2)*(99 + 99x + 2x^2) + (98 + 0x + 98x^2) \\
  &= (2772 + 8316x + 5699x^2 + 211x^3 + 2x^4) + (98 + 0x + 98x^2) \\
  &= 2870 + 8316x + 5797x^2 + 211x^3 + 2x^4 \mod f \\
  &= 2870 + 8316x + 5797x^2 - 211 - 2x \\
  &= 2659 + 8314x + 5797x^2 \mod q \\
  &= 59 + 14x + 97x^2 \\
\\
v &= b \cdot r + e_2 + (m*\left\lfloor q/2\right\rfloor) \\
  &= (73 + 54x + 31x^2) * (99 + 99x + 2x^2) + (1 + 0x + 0x^2) +  (1+ 1x+ 0x^2)*(100/2) \\
  &= (7227 + 12573x + 8561x^2 + 3177x^3 + 62x^4) + (1 + 0x + 0x^2) +  (50+ 50x+ 0x^2) \\
  &= 7278 + 12623x + 8561x^2 + 3177x^3 + 62x^4 \mod f \\
  &= 7278 + 12623x + 8561x^2 - 3177 - 62x \\
  &= 4101 + 12561x + 8561x^2 \mod q \\
  &= 1 + 61x + 61x^2  
\end{align*}
$$

Zum entschlüsseln kann man dann die selbe Formel wie vorher verwendet:

$$
\begin{align*}
m &= \left\lfloor \frac{1}{\left\lfloor q/2\right\rfloor}*(v-s^T \cdot u)\right\rceil _2 \\
  &= \left\lfloor \frac{1}{\left\lfloor 100/2\right\rfloor}*((1 + 61x + 61x^2 )- (1 + 0x + 1x^2) \cdot (59 + 14x + 97x^2))\right\rceil _2 \\
  &= \left\lfloor \frac{1}{50}*((1 + 61x + 61x^2 )- (59 + 14x + 156x^2 + 14.0x^3 + 97x^4))\right\rceil _2 \\
  &= \left\lfloor \frac{1}{50}*(-58 + 47x - 95x^2 - 14x^3 - 97x^4)\right\rceil _2 \mod f \\
  &= \left\lfloor \frac{1}{50}*(-58 + 47x - 95x^2 + 14 + 97x)\right\rceil _2  \\
  &= \left\lfloor \frac{1}{50}*(-44 + 144x - 95x^2)\right\rceil _2 \mod q \\
  &= \left\lfloor \frac{1}{50}*(56 + 44x + 5x^2)\right\rceil _2 \\
  &= \left\lfloor 1.12 + 0.88x + 0.1x^2\right\rceil _2 \\
  &= 1 + 1x + 0x^2 \\
\end{align*}
$$

Und daraus lässt sich die originale Nachricht auslesen $m' = 1 + 1x + 0x^2 \Rightarrow (1, 1, 0) $

### Code

In [11]:
from functools import partial
from numpy.polynomial import Polynomial
from numpy.random import randint, normal
import numpy as np

def polymod(a: Polynomial, f: Polynomial, q:int)->Polynomial:
    return Polynomial((a%f).coef%q)

def polymul(a: Polynomial,b: Polynomial, f:Polynomial, q: int)->Polynomial:
    return polymod(a*b, f, q)

def polyadd(a:Polynomial,b:Polynomial,f:Polynomial,q:int)->Polynomial:
    return polymod(a+b, f, q)

def sample_small_poly(n, q, sigma=1.0)->Polynomial:
  """Samples small vector from an approximated discrete Gaussian distribution.
  """
  # Sample from continuous Gaussian with mean 0 and std dev sigma
  error = np.round(normal(loc=0, scale=sigma, size=n)).astype(int) % q 

  # if its the zero vector, try again
  if sum(error) == 0:
    return sample_small_poly(n, q, sigma)
  return Polynomial(error)


In [12]:
modulus = 100
q_half = np.floor(modulus/2)
t = 2
m_p = 3
f = Polynomial([1]+[0]*(m_p-1)+[1])

pmul = partial(polymul, f=f, q=modulus)
padd = partial(polyadd, f=f, q=modulus)

s = sample_small_poly(m_p, modulus)
A = Polynomial(randint(0, modulus, m_p))
e = sample_small_poly(m_p, modulus)
b = padd(pmul(A, s), e)

print(f"""Secret Key: {s}
Private Key:
      A={A},
      b={b}
Generated with error: {e}""")

Secret Key: 0.0 + 0.0·x + 99.0·x²
Private Key:
      A=31.0 + 15.0·x + 93.0·x²,
      b=14.0 + 92.0·x + 70.0·x²
Generated with error: 99.0 + 99.0·x + 1.0·x²


In [13]:
message = randint(0, t, m_p) # random 0 or 1 as message
r = sample_small_poly(m_p, modulus)
e1 = sample_small_poly(m_p, modulus)
e2 = sample_small_poly(m_p, modulus)

u = padd(pmul(A, r), e1)
v = padd(padd(pmul(b, r), e2), message*q_half)

print(f"""Encrypting message: {message}
using:
    r={r},
    e1={e1}
    e2={e2}
Resulting in:
    u={u}
    v={v}
""")

decrypt = np.round((1/q_half * padd(v,-pmul(s, u)).coef)) % t
print(f"""Decrypts into message: {decrypt}""")

Encrypting message: [0 0 0]
using:
    r=1.0 + 2.0·x + 99.0·x²,
    e1=0.0 + 99.0·x + 99.0·x²
    e2=99.0 + 1.0·x + 1.0·x²
Resulting in:
    u=60.0 + 69.0·x + 91.0·x²
    v=65.0 + 91.0·x + 41.0·x²

Decrypts into message: [0. 0. 0.]


## M-LWE

### Code

In [58]:
import numpy as np
from numpy.random import randint, normal
from scipy.linalg import circulant


class ModuleMat:

    def __init__(self, poly: np.ndarray, modulus: int):
        self.poly = poly
        self.modulus = modulus

    @property
    def shape(self):
        return self.poly.shape[:-1], self.poly.shape[-1]

    def transpose(self) -> "ModuleMat":
        return ModuleMat(np.transpose(self.poly, (1, 0, 2)),
                         self.modulus) if self.poly.ndim == 3 else self

    @property
    def T(self) -> "ModuleMat":
        return self.transpose()

    @staticmethod
    def random_polynomial(
        poly_len: int,
        modulus: int,
        min_val: int = 0,
        max_value: int = None,
        shape: tuple = ()) -> "ModuleMat":
        """
        Generate an random polynomial of shape in the ring R^{rows x cols}_{modulus} with values
        evenly retrieved from min_val to max_val
        """
        if max_value is None:
            max_value = modulus
        assert all([s > 0 for s in shape]) and poly_len > 0
        return ModuleMat(randint(min_val, max_value, (*shape, poly_len)) % modulus,
                         modulus)

    def __repr__(self) -> str:
        return repr(self.poly)

    def __add__(self, other: "ModuleMat") -> "ModuleMat":
        assert self.shape == other.shape and self.modulus == other.modulus
        return ModuleMat((self.poly + other.poly) % self.modulus, self.modulus)

    def __sub__(self, other: "ModuleMat") -> "ModuleMat":
        assert self.shape == other.shape and self.modulus == other.modulus
        return ModuleMat((self.poly - other.poly) % self.modulus, self.modulus)

    def _generate_multiplication_matrix(self, polys: np.ndarray) -> np.ndarray:
        assert polys.ndim > 1, "Not Implemented Error"
        if polys.ndim == 2:
            rows = 1
            cols = polys.shape[0]
        else:
            rows, cols = polys.shape[0:2]
        blocks = [
            circulant(vec) * ((np.tri(polys.shape[-1]) * 2) - 1)
            for vec in polys.reshape(-1, (polys.shape[-1]))
        ]
        return np.vstack(
            [np.hstack(blocks[i * cols:(i + 1) * cols]) for i in range(rows)])

    def __matmul__(self, other: "ModuleMat") -> "ModuleMat":
        assert self.shape[1] == other.shape[1] and self.modulus == other.modulus
        assert self.modulus == other.modulus
        end_shape = (1, self.shape[1]) if self.poly.ndim == 2 else (
            self.poly.shape[0], self.shape[1])
        
        mm = self._generate_multiplication_matrix(self.poly)
        return ModuleMat(
            (mm  @ other.poly.flatten()).reshape(end_shape) % self.modulus,
            self.modulus)


In [59]:
poly1 = ModuleMat.random_polynomial(3, 100, shape=(2,))
poly2 = ModuleMat.random_polynomial(3, 100, shape=(2,))

In [60]:
print(poly1)
print(poly2)
print("---")
poly1 @ poly2

array([[55, 74, 67],
       [90, 80, 10]])
array([[65, 11, 55],
       [53, 75, 67]])
---


array([[28., 50., 54.]])

In [145]:
poly_len = 256
modulus = 3000
rows = 3
cols = 3

In [146]:
s = ModuleMat.random_polynomial(poly_len, modulus, min_val=-3, max_value=3, shape=(rows,))
A = ModuleMat.random_polynomial(poly_len, modulus, min_val=0, max_value=modulus, shape=(rows, cols))
e = ModuleMat.random_polynomial(poly_len, modulus, min_val=-3, max_value=3, shape=(cols,))

b = A @ s + e
pk = (A, b)

In [147]:
m = np.asarray([randint(0,2, poly_len)])
m_p = ModuleMat(m*(modulus//2), modulus)

r = ModuleMat.random_polynomial(poly_len, modulus, min_val=-3, max_value=3, shape=(rows,))
e1 = ModuleMat.random_polynomial(poly_len, modulus, min_val=-3, max_value=3, shape=(cols,))
e2 = ModuleMat.random_polynomial(poly_len, modulus, min_val=-3, max_value=3, shape=(1,))

u = (A.T @ r) + e1
v = b.T @ r + e2 + m_p


In [148]:
m_d = np.rint((v - (s.T @ u)).poly * (1/(modulus//2))) % 2

In [149]:
assert all(m_d.astype(int)[0] == m[0])